# PolymerXtal - Polymer Crystal Structure Generator and Analysis Software

In [ ]:
from hublib import use
from hublib.cmd import runCommand

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

polymer_type:
    type: Text
    value: PAN
    description: Select the polymer type.
    options: [PAN, PE, PP, PS, POM, PTFE, PVC]

helicity:
    type: Text
    value: Helix 2*3/1
    description: Select the helicity of the polymer chain.
    options: [Helix 2*3/1, Helix 2*2/1, Helix 2*1/1]
    
monomers:
    type: Number
    description: Number of monomers
    value: 30
    min: 1
    max: 100

tacticity:
    type: Text
    value: isotactic
    description: Select the tacticity of the polymer chain.
    options: [isotactic, atactic, syndiotactic, None]

chiriality:
    type: Text
    value: right
    description: Select the chiriality of the polymer chain.
    options: [left, right, None]

head_tail_defect_ratio:
    type: Number
    description: Ratio of head-to-head and tail-to-tail connections
    value: 0
    min: 0
    max: 1
        
configs:
    type: Number
    description: Number of attempts to find a configuration
    value: 30
    min: 1
    max: 5000

infinite:
    type: Boolean
    description: Whether to create periodic infinite chain
    value: True
        
create_lmpdata_file:
    type: Boolean
    description: Whether to create corresponding LAMMPS data file or not
    value: True

create_lmpinput_file:
    type: Boolean
    description: Whether to create corresponding LAMMPS input File or not
    value: True    

ffield:
    type: Text
    value: Dreiding
    description: Force field
    options: [Dreiding, PCFF]

bondscale:
    type: Number
    description: Applied to equilibrium bond lengths
    value: 1.1
    min: 1
    max: 2

charge:
    type: Text
    value: Gasteiger
    description: Charge equilibration method
    options: [Gasteiger]

In [ ]:
polymer_type = "PAN"
helicity = "Helix 2*3/1"
monomers = 30
tacticity = "isotactic"
chiriality = "right"
head_tail_defect_ratio = 0
configs = 30
infinite = True
create_lmpdata_file = True
create_lmpinput_file = True
ffield = "Dreiding"
bondscale = 1.1
charge = "Gasteiger"

In [ ]:
EXTRA_FILES = ['polymerxtal']

In [ ]:
import os
import sys
from polymerxtal import Helice, Chain
from simtool import DB

In [ ]:
helix=helicity.split()[1]
helice_atoms=eval(helix.split('*')[0])
helice_motifs=eval(helix.split('*')[1].split('/')[0])
helice_turns=eval(helix.split('/')[1])
chain=Chain(polymer_type=polymer_type,helice=Helice(helice_atoms,helice_motifs,helice_turns),num_monomers=monomers,tacticity=tacticity,
            chiriality=chiriality,head_tail_defect_ratio=head_tail_defect_ratio,configs=configs,infinite=infinite)
helix_name=chain.build_chain(use_visualize=True, create_lmpdata_file=create_lmpdata_file, create_lmpinput_file=create_lmpinput_file,ffield=ffield,
                             bondscale=bondscale,charge=charge)

In [ ]:
%%yaml OUTPUTS
HelixName:
    type: Text
    
PDBview:
    type: Text 
        
PDB:
    type: Text

LAMMPSDataFile:
    type: Text

Datafile_warnings:
    type: Text
        
X6paircoeffs:
    type: Text

LAMMPSinputfile:
    type: Text

In [ ]:
def copyAndSaveFileAsOutput(outputVariableName,inputPath):
    if inputPath.startswith('file://'):
        resultFile = os.path.basename(inputPath[7:])
        if resultFile != inputPath[7:]:
            shutil.copy2(inputPath[7:],resultFile)
    else:
        resultFile = os.path.basename(inputPath)
        if resultFile != inputPath:
            shutil.copy2(inputPath,resultFile)
    db.save(outputVariableName,file=resultFile)

In [ ]:
db = DB(OUTPUTS)
print(db)
db.save('HelixName', helix_name)
copyAndSaveFileAsOutput('PDBview', helix_name+'_view.pdb')
copyAndSaveFileAsOutput('PDB', helix_name+'.pdb')
if os.path.exists(helix_name+'.data'):
    copyAndSaveFileAsOutput('LAMMPSDataFile', helix_name+'.data')
if os.path.exists('Datafile_warnings.txt'):
    copyAndSaveFileAsOutput('Datafile_warnings', 'Datafile_warnings.txt')
if os.path.exists('X6paircoeffs.txt'):
    copyAndSaveFileAsOutput('X6paircoeffs', 'X6paircoeffs.txt')
if os.path.exists('LAMMPSinputfile.txt'):
    copyAndSaveFileAsOutput('LAMMPSinputfile', 'LAMMPSinputfile.txt')